In [2]:
import os
import pandas as pd
import re
import json
from nltk.corpus import stopwords
import nltk
import spacy


nltk.download('punkt')
nltk.download('stopwords')


class TextPreprocessor:
    def __init__(self):
        self.stop_phrases = [
            'Ещё больше новостей — в телеграм-канале Москва 24 Подписывайтесь!', 
            'Подробнее – в эфире телеканала Москва 24.', 
            'РИА «Сахалин-Курилы»'
        ]

        self.tags_keyword = '\nTags: '

        self.word_with_capital_letter_pattern = re.compile('^[А-ЯA-Z]')
        self.sentence_ending_char_on_new_line_pattern = re.compile('(?<=[а-я])\n(?=\\.)')
        self.footnote_pattern = re.compile('^\\*+')
        self.citation_beginning_pattern = re.compile('^("|“|«)')
        self.two_spaces_pattern = re.compile('\\s{2,}')
        self.two_quotes_pattern = re.compile('"{2,}')
        self.ria_news_reference_pattern = re.compile('\\b[А-Я]+, \\d{1,2} [а-я]+ — РИА Новости\\b')
        self.website_pattern = re.compile('\\b(https?://)?[a-zA-Z\\d][\\w\\-]*(\\.[a-zA-Z\\d][\\w\\-]*[a-zA-Z\\d])*\\.[\\w\\-]*[a-zA-Z\\d]\\b')
        self.visual_credit_pattern = re.compile('\\b(Видео|Фото):')

        self.stop_words = stopwords.words('russian')
        self.stop_words.extend(['это', 'эта', 'этот', 'ещё', 'которые', 'который', 'которым', 'также'])

        self.russian_nlp = spacy.load('ru_core_news_lg')


    def _preprocess_tags(self, tags: list[str]):
        _tags = [t.lower().replace('#', '') for t in tags]

        _tags = [t for t in _tags if len(t.split(' ')) == 1]

        return _tags
    

    def _tokenize_text(self, text: str):
        document = self.russian_nlp(text)

        tokens = []

        for token in document:
            t = token.lemma_.lower()

            if (t in self.stop_words) or not t.isalpha():
                continue

            tokens.append(t)

        return tokens


    def preprocess_text(self, text: str):
        # There is a key word "Tags:" in some news items. This word marks 
        # the beginning of a sequence of tags that were assigned to a corresponding 
        # news message. These tags are irrelevant for the topic modeling of the corpus, so we delete them
        text = text.strip('"')
        text = re.sub(self.two_quotes_pattern, '"', text)

        if self.tags_keyword in text:
            _text, tags_sequence = text.split(self.tags_keyword)

            tags = [t.strip(' ').replace('#', '') for t in tags_sequence.split(',')]

            # We preprocess tags to use them later as 
            # ground truth in the topic modeling quality estimation
            tags = self._preprocess_tags(tags)
        else:
            _text = text + ''

            tags = []

        lines = _text.split('\n')

        relevant_lines = []
        footnotes = []

        for l in lines:
            l = l.strip(' \t')

            if len(l) == 0:
                continue
            
            # We identify lines of text that are footnotes and set them aside of a news message text
            if bool(self.footnote_pattern.search(l)):
                footnotes.append(l)
            else:
                relevant_lines.append(l)

        _text = ' '.join(relevant_lines)
        _text = re.sub(self.two_spaces_pattern, ' ', _text)

        # We delete stop phrases from the text
        for phi in self.stop_phrases:
            _text = _text.replace(phi, '')

        _text = re.sub(self.ria_news_reference_pattern, '', _text)
        _text = re.sub(self.visual_credit_pattern, '', _text)
        _text = re.sub(self.website_pattern, '', _text)

        _text = _text.strip()

        tokens = self._tokenize_text(_text)

        return _text, tokens, tags, footnotes
    

tatar_specific_characters = set('ӘәҖҗӨөҢңҺһ')


def tatar_characters_present(text: str):
    text_char_set = set(text)

    return len(tatar_specific_characters & text_char_set) > 0

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mdmytriiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mdmytriiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
root_path = 'datasets'

filename = 'processed_test_assignment_data.csv'

raw_data = pd.read_csv(os.path.join(root_path, filename))

preprocessor = TextPreprocessor()

X = []
processed_texts = []
processed_dataset = []

for i, row in raw_data.iterrows():
    if tatar_characters_present(row['fullText']):
        continue
    
    text, tokens, tags, footnotes = preprocessor.preprocess_text(row['fullText'])

    processed_dataset.append({
        'text': text, 
        'tokens': tokens, 
        'tags': tags, 
        'footnotes': footnotes
    })

    processed_texts.append(text)
    X.append(tokens)

with open(os.path.join(root_path, 'processed_dataset.json'), 'w') as f:
    json.dump(processed_dataset, f)

In [4]:
processed_texts[:10]

['Бабушкинский суд столицы приговорил трех активистов движения "СтопХам" к шести годам тюремного заключения каждого за драку с сотрудниками спецподразделения "Гром" в Москве. Об этом сообщается в телеграм-канале судов общей юрисдикции столицы.Там уточнили, что суд признал виновными Кирилла Бунина, Кирилла Котова и Алексея Горбачевского. Они обвиняются в применении насилия в отношении представителя власти (статья 318 УК РФ) и в совершенном группой лиц хулиганстве (часть 2 статьи 213 УК РФ). Отбывать наказание мужчины будут в колонии общего режима, добавили там. Конфликт между активистами и правоохранителями произошел на Ярославском шоссе 23 июня 2022 года из-за неправильно припаркованной машины сотрудников спецотряда. Отмечалось, что обвиняемые ругались и били по капоту машины, после чего началась драка.Спустя два дня Бабушкинский суд Москвы арестовал нападавших до 23 августа 2022 года. В июле 2023-го в столице начался процесс над активистами "СтопХама".',
 'Бывшего совладельца сетей "К

In [5]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvisualize

In [9]:
# load dictionary
dictionary = corpora.Dictionary(X)
dictionary.filter_extremes(no_below=5, no_above=0.9)

# generate corpus as BoW
corpus = [dictionary.doc2bow(tokens_list) for tokens_list in X]

num_topics = 50

# train LDA model
lda_model = LdaModel(corpus=corpus, 
                     id2word=dictionary, 
                     random_state=4583, 
                     chunksize=200, 
                     num_topics=num_topics, 
                     passes=200, 
                     iterations=400)

# print LDA topics
for topic in lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(topic)

lda_model.save('topic_model/lda_model')

(0, '0.067*"служба" + 0.050*"пресс" + 0.046*"область" + 0.036*"сообщить" + 0.035*"февраль" + 0.024*"произойти" + 0.022*"сообщать" + 0.019*"ранее" + 0.019*"водитель" + 0.018*"ведомство"')
(1, '0.062*"год" + 0.049*"регион" + 0.045*"область" + 0.038*"работа" + 0.038*"проект" + 0.031*"губернатор" + 0.026*"развитие" + 0.021*"программа" + 0.021*"правительство" + 0.018*"образование"')
(2, '0.360*"новость" + 0.315*"риа" + 0.024*"советник" + 0.024*"ветер" + 0.023*"источник" + 0.016*"национальный" + 0.014*"уделять" + 0.013*"предупреждать" + 0.013*"reuters" + 0.012*"ряд"')
(3, '0.049*"температура" + 0.044*"февраль" + 0.037*"роман" + 0.036*"неделя" + 0.036*"день" + 0.035*"градус" + 0.035*"снег" + 0.033*"ночь" + 0.031*"воздух" + 0.028*"ожидаться"')
(4, '0.302*"около" + 0.178*"планировать" + 0.106*"транспортный" + 0.092*"построить" + 0.080*"инфраструктура" + 0.062*"трасса" + 0.060*"километр" + 0.020*"автодорога" + 0.009*"дорожно" + 0.003*"варшавский"')
(5, '0.072*"россия" + 0.050*"страна" + 0.034*"г

In [21]:
with open('datasets/processed_dataset.json', 'r') as f:
    processed_dataset = json.load(f)

X = [item['tokens'] for item in processed_dataset]

In [10]:
coherence_model = CoherenceModel(model=lda_model, texts=X, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(coherence_score)

0.4546773645184921


In [11]:
russian_news_topics_visualization = gensimvisualize.prepare(lda_model, corpus, dictionary, mds='mmds')
pyLDAvis.display(russian_news_topics_visualization)